In [1]:
import joblib
import pretty_midi
import pandas as pd
import numpy as np
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor

2021-12-01 20:16:12.242560: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-01 20:16:12.242664: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Getting data

## Simple dataset

In [2]:
X, y_pitch, y_velocity = joblib.load(f'../raw_data/pandas_dataframes/simple_dataset/nparray0')
for i in range(1, 10):
    loaded = joblib.load(f'../raw_data/pandas_dataframes/simple_dataset/nparray{i}')
    X = np.concatenate((X, loaded[0]))
    y_pitch = np.concatenate((y_pitch, loaded[1]))
    y_velocity = np.concatenate((y_velocity, loaded[2]))
    print(f'{i} done')

1 done
2 done
3 done
4 done
5 done
6 done
7 done
8 done
9 done


In [3]:
X.shape

(77, 500, 128)

In [4]:
X_train, X_test, y_pitch_train, y_pitch_test, y_velocity_train, y_velocity_test = train_test_split(X, y_pitch, y_velocity, test_size=0.3)

## Bigger Dataset

In [23]:
X, y_pitch, y_velocity = joblib.load(f'../raw_data/pandas_dataframes/bigger_dataset/nparray0')
for i in range(1, 50):
    loaded = joblib.load(f'../raw_data/pandas_dataframes/bigger_dataset/nparray{i}')
    X = np.concatenate((X, loaded[0]))
    y_pitch = np.concatenate((y_pitch, loaded[1]))
    y_velocity = np.concatenate((y_velocity, loaded[2]))
    print(f'{i} done')

1 done
2 done
3 done
4 done
5 done
6 done
7 done
8 done
9 done
10 done
11 done
12 done
13 done
14 done
15 done
16 done
17 done
18 done
19 done
20 done
21 done
22 done
23 done
24 done
25 done
26 done
27 done
28 done
29 done
30 done
31 done
32 done
33 done
34 done
35 done
36 done
37 done
38 done
39 done
40 done
41 done
42 done
43 done
44 done
45 done
46 done
47 done
48 done
49 done


# Testing DL models

## LSTM - Classic - Regression

In [38]:
inputs = layers.Input(shape=(500, 128)) # TO BE MODIFIED - SPARSE MATRIX  #input shape 128 * 10_000

# Envoi d'un sparse tensor sinon sparse matrix
# Envoyer en input sparse ou brut est-ce que ça change les perfs ou non
# https://stackoverflow.com/questions/20459536/convert-pandas-dataframe-to-sparse-numpy-matrix-directly

main_branch = layers.Masking(mask_value=0)(inputs)
main_branch = layers.LSTM(units=128, activation='tanh', return_sequences=True)(main_branch)
main_branch = layers.LSTM(units=128, activation='tanh')(main_branch)
main_branch = layers.Dense(64, activation='relu')(main_branch)
# DO WE NEED TWO LAYERS FOR REGRESSION AS WELL ?
pitch_branch = layers.Dense(500, activation='relu', name='pitch_output')(main_branch)
velocity_branch = layers.Dense(500, activation='relu', name='velocity_output')(main_branch)

model_lstm_reg = models.Model(inputs = inputs, outputs = [pitch_branch, velocity_branch])

In [39]:
model_lstm_reg.compile(loss='mse', optimizer='rmsprop') # LOSS TO REVIEW

In [40]:
model_lstm_reg.fit(X_train,
              {'pitch_output': y_pitch_train, 'velocity_output': y_velocity_train},
              epochs=3,
              batch_size=8,
              verbose=1,
              validation_split=0.2,
             )

Epoch 1/3
6/6 [==============================] - 14s 1s/step - loss: 9706.4238 - pitch_output_loss: 3175.2544 - velocity_output_loss: 6531.1694 - val_loss: 10705.6494 - val_pitch_output_loss: 3243.4358 - val_velocity_output_loss: 7462.2129
Epoch 2/3
6/6 [==============================] - 5s 779ms/step - loss: 9401.3330 - pitch_output_loss: 3048.1240 - velocity_output_loss: 6353.2085 - val_loss: 10031.0908 - val_pitch_output_loss: 2971.9373 - val_velocity_output_loss: 7059.1533
Epoch 3/3
6/6 [==============================] - 5s 792ms/step - loss: 8691.8584 - pitch_output_loss: 2756.4509 - velocity_output_loss: 5935.4072 - val_loss: 9142.1377 - val_pitch_output_loss: 2628.4707 - val_velocity_output_loss: 6513.6670


In [41]:
to_predict = joblib.load('../raw_data/pandas_dataframes/bigger_dataset/nparray28')[0]
true_pitches = joblib.load('../raw_data/pandas_dataframes/bigger_dataset/nparray28')[1]
true_velocities = joblib.load('../raw_data/pandas_dataframes/bigger_dataset/nparray28')[2]

In [42]:
to_predict[0].shape

(500, 128)

In [43]:
predictions = model_lstm_reg.predict(to_predict)

In [44]:
len(predictions), predictions[0].shape, predictions[1].shape

(2, (12, 500), (12, 500))

In [46]:
for i in range(12):
    print(f'prediction {i+1}')
    print(predictions[1][i])
    print(predictions[1][i].mean())
    print(f'true velocities {i+1}')
    print(true_velocities[i])

prediction 1
[ 0.         14.448837    8.772736    9.175815    7.1733527  11.563968
  9.174725    7.375934    0.          9.766924    9.374606    0.
  5.6858253   8.308912    9.691753    7.6841984  11.738454    9.960739
 13.725515    5.804236    8.772683    7.2288494   0.          6.548243
  9.436388    8.912844   12.319378   11.77016     0.         10.378804
  5.3871584   7.7704754   9.303396    7.637772    3.3219347   6.295297
  8.808726    8.796823    9.508945    6.831757    8.321323    7.7743936
  9.778733   11.111731    8.629077    8.5852995   4.5612407   8.771295
  6.540793    9.7514925   9.708191    9.524297    0.          0.
  8.540388    9.536207    8.168704    9.366955   10.001665    7.9070773
  0.         10.637432    8.469469    7.7779965   7.8657618   9.719302
  0.          9.921306    7.670378    8.018879    0.          6.534943
  8.691263    7.393707    7.7114363  12.972172    7.103353    7.417578
 13.359323   10.092494    8.562441    5.247567    0.          7.473106
  0

In [ ]:
def piano_roll_from_targets(pitches, velocities):
    
    piano_roll = np.array()
    for pitch in pitches:
        piano_roll[int(pitch)]

## Biderectional

In [21]:
inputs = layers.Input(shape=(128, 10_000), name='main_input') # TO BE MODIFIED - SPARSE MATRIX
# Add 2 bidirectional LSTMs
main_branch = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(main_branch)
main_branch = layers.Bidirectional(layers.LSTM(64))(main_branch)
# DO WE NEED TWO LAYERS FOR REGRESSION AS WELL ?
pitch_branch = layers.Dense(1, activation='linear', name='pitch_output')(main_branch)
velocity_branch = layers.Dense(1, activation='linear', name='velocity_output')(main_branch)
model_lstm_bin_reg = models.Model(inputs = inputs, outputs = [pitch_branch, velocity_branch])
model_lstm_bin_reg.summary()

ValueError: Input 0 of layer "bidirectional_1" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (None, 64)

# Testing ML Random Forests

In [36]:
decision_tree_regressor = DecisionTreeRegressor()
X_train = X_train.reshape((53, 500 * 128))
decision_tree_regressor.fit(X_train, y_pitch_train)

DecisionTreeRegressor()

In [37]:
y_pitch_pred = decision_tree_regressor.predict(X_test[9].reshape(1, -1))

In [38]:
decision_tree_regressor = DecisionTreeRegressor()

In [39]:
decision_tree_regressor.fit(X_train, y_velocity_train)

DecisionTreeRegressor()

In [42]:
y_velocity_pred = decision_tree_regressor.predict(X_test[9].reshape(1, -1))

In [43]:
y_pitch_pred.shape

(1, 500)

In [58]:
# Let's listen to this
def melody_to_piano_roll(pitches, velocities):
    """
    Create a piano roll from a list of pitches and a list of velocities
    """
    piano_roll = np.zeros((128, pitches.shape[1]))
    
    for i, (pitch, velocity) in enumerate(zip(pitches[0], velocities[0])):
        if pitch > 0:
            piano_roll[int(pitch)][i] = velocity
            
    return piano_roll

In [59]:
melody = melody_to_piano_roll(y_pitch_pred, y_velocity_pred)

In [61]:
from MIDIComposingAI.utils import piano_roll_to_pretty_midi

In [62]:
pretty_melody = piano_roll_to_pretty_midi(melody, fs=50)

In [64]:
pretty_melody.write('melody_predicted.mid')